<a href="https://colab.research.google.com/github/ZombiePy/MatrixRoadSign/blob/master/day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/MatrixRoadSign/MatrixRoadSign'

/content/drive/My Drive/Colab Notebooks/MatrixRoadSign/MatrixRoadSign


In [0]:
ls

data/  day2.ipynb  LICENSE  README.md


In [0]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1 : y_train = to_categorical(y_train)
if y_test.ndim == 1 : y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
model = Sequential([
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=input_shape),

                    Flatten(),

                    Dense(num_classes, activation='softmax')
                    ])

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.fit(X_train, y_train)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
flatten (Flatten)            (None, 57600)             0         
_________________________________________________________________
dense (Dense)                (None, 43)                2476843   
Total params: 2,478,635
Trainable params: 2,478,635
Non-trainable params: 0
_________________________________________________________________
1088/1088 [==============================] - 4s 4ms/step - loss: 19.4049 - accuracy: 0.7855


In [0]:
def get_cnn_v1(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=input_shape),

                    Flatten(),

                    Dense(num_classes, activation='softmax')
                    ])

def train_model(model, X_train, y_train, params_fit={}):

  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_test, y_test)),
      callbacks=[tensorboard_callback]
      )
  return logdir
   

In [0]:
logdir = train_model(get_cnn_v1(input_shape, num_classes), X_train, y_train)

Epoch 1/5
272/272 [==============================] - 3s 11ms/step - loss: 40.2134 - accuracy: 0.7302 - val_loss: 0.2318 - val_accuracy: 0.9473
Epoch 2/5
272/272 [==============================] - 3s 10ms/step - loss: 0.2195 - accuracy: 0.9508 - val_loss: 0.2557 - val_accuracy: 0.9552
Epoch 3/5
272/272 [==============================] - 3s 10ms/step - loss: 0.1199 - accuracy: 0.9717 - val_loss: 0.0793 - val_accuracy: 0.9804
Epoch 4/5
272/272 [==============================] - 3s 10ms/step - loss: 0.0917 - accuracy: 0.9776 - val_loss: 0.0833 - val_accuracy: 0.9819
Epoch 5/5
272/272 [==============================] - 3s 10ms/step - loss: 0.0958 - accuracy: 0.9791 - val_loss: 0.0762 - val_accuracy: 0.9812


In [0]:
%tensorboard --logdir logs

In [0]:
def get_cnn_v2(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                    
                    MaxPool2D(),
                    
                    Dropout(0.5),
                    
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                    
                    MaxPool2D(),
                    
                    Dropout(0.3),
                    
                    Flatten(),
                     
                    Dense(1024, activation='relu'),
                     
                    Dropout(0.5),

                    Dense(num_classes, activation='softmax')
                    ])

ERROR! Session/line number was not unique in database. History logging moved to new session 60


In [0]:
params_fit = {
    'epochs': 25
}
logdir = train_model(get_cnn_v2(input_shape, num_classes), X_train, y_train, params_fit)

Epoch 1/25
272/272 [==============================] - 3s 11ms/step - loss: 8.3885 - accuracy: 0.1305 - val_loss: 2.8058 - val_accuracy: 0.2312
Epoch 2/25
272/272 [==============================] - 3s 10ms/step - loss: 2.2585 - accuracy: 0.3636 - val_loss: 1.3156 - val_accuracy: 0.6434
Epoch 3/25
272/272 [==============================] - 3s 10ms/step - loss: 1.2491 - accuracy: 0.6156 - val_loss: 0.5739 - val_accuracy: 0.8534
Epoch 4/25
272/272 [==============================] - 3s 11ms/step - loss: 0.8352 - accuracy: 0.7384 - val_loss: 0.3494 - val_accuracy: 0.9201
Epoch 5/25
272/272 [==============================] - 3s 10ms/step - loss: 0.6335 - accuracy: 0.8003 - val_loss: 0.2407 - val_accuracy: 0.9509
Epoch 6/25
272/272 [==============================] - 3s 10ms/step - loss: 0.5283 - accuracy: 0.8320 - val_loss: 0.1607 - val_accuracy: 0.9671
Epoch 7/25
272/272 [==============================] - 3s 11ms/step - loss: 0.4567 - accuracy: 0.8575 - val_loss: 0.1458 - val_accuracy: 0.9648